In [1]:
import geocoder
import unicodecsv
import logging
import time
import csv
from pygeocoder import Geocoder
from uszipcode import ZipcodeSearchEngine
import pandas as pd
import blaze
import time
import os

/anaconda3/lib/python3.6/site-packages/odo/backends/pandas.py:102: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))


In [2]:
# parse_dates = true to read dates automatically
divvyTrips =pd.read_csv('Social_Economic_Info_By_Zipcode/MergedData/Divvy_2016_merged.csv',parse_dates=True)


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (45,46,47,48,66,70) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
divvyTrips.stoptime=pd.to_datetime(divvyTrips.stoptime)
divvyTrips.starttime=pd.to_datetime(divvyTrips.starttime)
divvyTrips['Year'] = divvyTrips['starttime'].apply(lambda d: d.year)
divvyTrips['Month'] = divvyTrips['starttime'].apply(lambda d: d.month)
divvyTrips['Day'] = divvyTrips['starttime'].apply(lambda d: d.day)
divvyTrips['DayOfWeek'] = divvyTrips['starttime'].apply(lambda d: d.dayofweek)
divvyTrips['DayName'] = divvyTrips['starttime'].apply(lambda d: d.weekday_name)
divvyTrips['DayOfYear'] = divvyTrips['starttime'].apply(lambda d: d.dayofyear)
divvyTrips['WeekOfYear'] = divvyTrips['starttime'].apply(lambda d: d.weekofyear)
divvyTrips['Quarter'] = divvyTrips['starttime'].apply(lambda d: d.quarter)

In [4]:

#Remove all rows with missing values. axis =0 is the default 
divvyTrips = divvyTrips.dropna(subset=['trip_id'], how="all", axis=0)

import calendar
divvyTrips['month_name'] = divvyTrips['Month'].apply(lambda x: calendar.month_abbr[x])
divvyTrips['months-year']=divvyTrips['starttime'].apply(lambda x: x.strftime('%B-%Y'))

#months = {1: 'Jan', 2: 'Feb', 3: 'Mar',4: 'Apr',5:'May',6: 'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12: 'Dec'}
#divvyTrips['month_name'] = divvyTrips['Month'].map(months)
#divvyTrips['months-year'] = divvyTrips.reset_index().apply(lambda x: '{}-{}'.format(x['month_name'],x['Year']),axis=1)

#divvy_dataset['month-year'] = divvy_dataset.reset_index().apply(lambda x: '{}-{}'.format(x['months'],x['year']),axis=1)

#Convert Trip to minutes
divvyTrips['tripInMinutes'] = divvyTrips['tripduration'] / 60

#Add the Hour column
divvyTrips.starttime = pd.to_datetime(divvyTrips.starttime)
divvyTrips['Hour'] = divvyTrips.starttime.apply(lambda x: x.hour)

## If trip start time is greater than stop time, swap them
divvyTrips.loc[divvyTrips.starttime > divvyTrips.stoptime,['starttime','stoptime']] = divvyTrips.loc[divvyTrips.starttime > divvyTrips.stoptime,['stoptime','starttime']].values

#Remove Dependent user type
divvyTrips = divvyTrips[divvyTrips.usertype != 'Dependent']

In [ ]:
## Group Station capacity into buckets

In [ ]:
#Convert an existing Series or column to a category dtype:
divvyTrips['gender'].astype('category')

int(divvyTrips.dpcapacity_start.max())
##### 55

labels = ["{0} - {1}".format(i, i + 9) for i in range(int(divvyTrips.dpcapacity_start.min()), int(divvyTrips.dpcapacity_start.max())*2, 10)]

#divvy_dataset['group'] = pd.cut(divvy_dataset.dpcapacity_start, range(int(divvy_dataset.dpcapacity_start.min()), int(divvy_dataset.dpcapacity_start.max()), 10), right=False, labels=labels)
divvyTrips['start_station_group'] = pd.cut(divvyTrips.dpcapacity_start, range(0, 120, 10), right=False, labels=labels)
divvyTrips.head()

divvyTrips['end_station_group'] = pd.cut(divvyTrips.dpcapacity_end, range(0, 120, 10), right=False, labels=labels)
divvyTrips.head()

stations_trips = divvyTrips.groupby('start_station_group')['trip_id','gender'].agg(len).dropna()

stations_trips.plot(kind='bar');

In [ ]:
## Add Holiday, weekday

In [5]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

cal = calendar()
holidays = cal.holidays(start=divvyTrips['starttime'].min(), end=divvyTrips['starttime'].max())
divvyTrips['TypeOfDay'] = divvyTrips['starttime'].dt.date.astype('datetime64').isin(holidays)
divvyTrips.loc[(divvyTrips.DayOfWeek == 5) | (divvyTrips.DayOfWeek == 6) | (divvyTrips.TypeOfDay == True),'TypeOfDay'] = 'Holiday'
divvyTrips.loc[(divvyTrips.TypeOfDay == False),'TypeOfDay'] = 'Weekday'

/anaconda3/lib/python3.6/site-packages/pandas/core/internals.py:3224: FutureWarning: Passing in 'datetime64' dtype with no frequency is deprecated and will raise in a future version. Please pass in 'datetime64[ns]' instead.
  return self.apply('astype', dtype=dtype, **kwargs)


In [ ]:
## Add Seasons to the dataframe

In [6]:
seasons = ['Winter', 'Winter', 'Winter', 'Spring', 'Spring', 'Summer', 'Summer', 'Summer', 'Autumn', 'Autumn', 'Autumn', 'Winter']
month_to_season = dict(zip(range(1,13), seasons))
divvyTrips['season']=divvyTrips.starttime.dt.month.map(month_to_season) 


In [7]:
# Drop Duplicates
divvyTrips = divvyTrips.drop_duplicates()

In [8]:
divvyTrips.to_csv('Social_Economic_Info_By_Zipcode/MergedData/Divvy_2016_merged.csv')